In [4]:
import openpyxl
import requests 
from bs4 import BeautifulSoup
import json
from urllib.parse import urlparse

setSQLData = []

def catchvalue(get):
    newword=str(get).replace("product_d.php?","").split('&')
    twnew=newword[0].replace("lang=","")  
    tbnew=newword[1].replace("tb=","")
    idnew=newword[2].replace("id=","")
    
    result={'lang':f'{twnew}',
            'tb':f'{tbnew}',
            'id':f'{idnew}',
            'type':'productdata'}
    return result
ths = ["產品名稱","產品描述","價格","尺寸","庫存","產品數量"]
def export(products):
    wb = openpyxl.Workbook()
    sheet = wb.create_sheet("各類鞋子產品明細",0)
    
    for th in ths
        sheet.append(th)
        for product in products:
            sheet.append(product)
        wb.save("shoes.xlsx")
#找每種類別的最終頁數

def allpage(f):      
    lst=[]
    for i in f:
        if "page_last" in str(i):            
            lst.append(i.find('a').get('href'))   #取到所有類別的最後頁數
            my_str=lst[0]
            my_str=my_str.replace("product.php?","")
            newstr=my_str.replace("product_d.php?","").split('&')
            new2=newstr[2].replace("cid=" , "")
            return new2
            
#高跟 cid =8 /16  
#低跟 cid=137 / 4
#涼鞋 cid=14 /3
#靴子 cid=129 /3
#平底 cid=11 /6
#男鞋 cid=121/3
#童鞋 cid=17/3

# 每個分類的cid 對應頁數
def Returnpage(cid):
    if cid =="8":
        return "16"    #頁數
    if cid =="137":
        return "4"
    if cid =="14":
        return "3"
    if cid =="129":
        return"3"
    if cid =="11":
        return "6"
    if cid =="121":
        return "3"
    if cid =="17":
        return "3"  
    

url ='https://www.momentanee.com/index.php?lang=tw'

re=requests.get(url)
soup=BeautifulSoup(re.text ,'html.parser')
headers = {
    'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
}

menu=soup.find('div',class_='dropdown-menu SP').find_all('a')
# print(menu)

diftype =dict()
for each in menu[1:8]:   #各種類
    title =each.text
    href = 'https://www.momentanee.com/'+ each.get('href')
    diftype[title]=href
#     print(title)
#     print(https://www.momentanee.com/'+ href)
        
    #各分類裡的資料
for title,href in diftype.items():
#     print(title,href)   
    eachsite=requests.get(href)
    sp=BeautifulSoup(eachsite.text,'html.parser')
#     print(sp)


#pages=sp.find('div', class_='page').findNext('select').find_all('option')   
      #同一品項總共頁數
    pages=sp.find('div',class_='page').findNext('ul').find_all('li')
#     print(pages)
    eachcid=allpage(pages)
#     print(eachcid)
    matchpage=Returnpage(eachcid)
#     print(matchpage)
       
    for j in range(1,int(matchpage)+1):  
        allurl=f"https://www.momentanee.com/product.php?lang=tw&tb=1&cid={eachcid}&ot=all&currentpage={j}"
        print(allurl)   #得到全部款式的所有頁的url

       
        request=requests.get(allurl)
        BS=BeautifulSoup(request.text , 'html.parser')
#         print(BS)
    
    #單一類別的所有鞋款
        product=BS.find('div', class_="panel_3 container" )
        items=product.find_all('div',class_='col-md-3 col-sm-4 col-xs-6')
#         print(items)
        info= []
        for i in items:
            result = 'https://www.momentanee.com/'+ i.find('a').get('href')           
#             print(result)  


            re = requests.get(result)
            sb = BeautifulSoup(re.text , 'html.parser')
        
 

            newhref=i.find('a').get('href')
#             print(newhref)
            catch=catchvalue(newhref)
#     #         print(catch)

            newweb=requests.post('https://www.momentanee.com/include/shopping_process.php',data = catch)
#             print(newweb)
            APItext=json.loads(newweb.text)      
            #print(APItext)

            
            if APItext['standskeys'] == []:
                heel=sb.find('div',class_='products-info')

                #名字
                name=heel.find('div',class_='name').text
                #print(name)
                
                
                #產品描述   
                if "editor clearfix" not in str(heel):
                    pass
                else:
                    describe=sb.find('article',class_="editor clearfix").text.replace('\n','').replace('門市資訊','').lstrip()
    #                print('產品描述:' , len(describe))


                    if len(describe) == 0:   #有空白
                        #print("產品描述:"  , "沒有資料")
                        describe = "沒有資料"
                    else:
                        #print("產品描述:"  , describe)
                        describe
                        
                price ="6800"    
                #print("售價:" ,price)
                size  = "40"
                #print("尺寸:" , size)
                stock = "2"
                #print("產品庫存:" , stock)
                productCount = "3"
                #print("數量:" , productCount)
                
                #print('\n')
                
                row = [name, describe, price, size, stock,productCount]
                setSQLData.append(row)
                
            else:
                for i in APItext['standskeys']:

                    heel=sb.find('div',class_='products-info')

                    #名字
                    name=heel.find('div',class_='name').text
                    #print(name)


                    #售價
        #             price= APItext['standskeys']   
                    price =  APItext['stands'][f'{i}']['price']
                    #print("售價:" , APItext['stands'][f'{i}']['price'])


                    #產品描述   
                    if "editor clearfix" not in str(heel):
                        pass
                    else:
                        describe=sb.find('article',class_="editor clearfix").text.replace('\n','').replace('門市資訊','').lstrip()
    #                     print('產品描述:' , len(describe))


                        if len(describe) == 0:   #有空白
                            #print("產品描述:"  , "沒有資料")
                            describe = "沒有資料"
                        else:
                            #print("產品描述:"  , describe)
                            describe


                    #尺寸
        #             size=APItext['standskeys']
                    size = APItext['stands'][f'{i}']['standName']
                    #print("尺寸:" , APItext['stands'][f'{i}']['standName'])

                    #產品庫存
        #             stock=APItext['standskeys']
                    stock = APItext['stands'][f'{i}']['inventory']
                    #print("產品庫存:" , APItext['stands'][f'{i}']['inventory'])

                    #數量
        #             quantity=APItext['standskeys']
                    productCount = APItext['stands'][f'{i}']['inventorySet']
                    #print("數量:" , APItext['stands'][f'{i}']['inventorySet'])
            
                    row = [name, describe, price, size, stock,productCount]
                    setSQLData.append(row)
                    #print('\n')


#export(setSQLData)
print(setSQLData)

SyntaxError: invalid syntax (<ipython-input-4-f6260ca3a7b8>, line 25)